# Ціль: Набуття навичок налаштування системи безпеки MS SQL Server


In [1]:
USE Accounting;

Commands completed successfully.

Total execution time: 00:00:00.001

In [3]:
--1. Створити новий обліковий запис;
CREATE LOGIN new_log1 WITH PASSWORD ='1234';
USE Accounting;
GO

CREATE USER new_log1 FOR LOGIN new_log1;
SELECT HAS_DBACCESS('Accounting') users;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.130

users
1


In [4]:
--2. Назначити користувачу роль БД (будь-яку);
EXECUTE sp_addrolemember db_datareader ,new_log1;

Commands completed successfully.

Total execution time: 00:00:00.162

In [5]:
--3. Надати користувачу Bill1 повноваження на доступ к збереженої процедури (будь-якої);
CREATE LOGIN Bill1 WITH PASSWORD='pass'
USE Accounting 
GO
CREATE USER Bill1 FOR LOGIN Bill1 
EXECUTE sp_addrolemember'db_datawriter','Bill1'
GRANT EXECUTE TO Bill1

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.045

In [6]:
--4. Відкликати всі надані користувачу John привілегії.
REVOKE ALL FROM Bill1;

The ALL permission is deprecated and maintained only for compatibility. It DOES NOT imply ALL permissions defined on the entity.

Total execution time: 00:00:00.025

In [7]:
--5. Зашифруйте зміст однієї з  колонок таблиці в створеній БД
GO 
USE master;
SELECT * from sys.symmetric_keys
WHERE name ='##MS_ServiceMasterKey##';

use Accounting;
CREATE MASTER KEY ENCRYPTION BY PASSWORD='1111';

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.285

name,principal_id,symmetric_key_id,key_length,key_algorithm,algorithm_desc,create_date,modify_date,key_guid,key_thumbprint,provider_type,cryptographic_provider_guid,cryptographic_provider_algid
##MS_ServiceMasterKey##,1,102,256,A3,AES_256,2019-12-02 20:14:03.880,2019-12-02 20:14:03.880,c769e0cc-21d6-48a6-90de-382a59e269b8,NULL,NULL,NULL,NULL


In [8]:
CREATE CERTIFICATE CertificateDBSP 
WITH SUBJECT='Protect Data';

CREATE SYMMETRIC KEY SymmetricKeyDBSP
WITH ALGORITHM=AES_128
ENCRYPTION BY CERTIFICATE CertificateDBSP;

Commands completed successfully.

Total execution time: 00:00:00.794

In [9]:
GO
USE accounting;
ALTER TABLE Students 
add surname_encrypt VARBINARY(max) NULL

Commands completed successfully.

Total execution time: 00:00:00.157

In [10]:
GO
OPEN SYMMETRIC KEY SymmetricKeyDBSP
DECRYPTION BY CERTIFICATE CertificateDBSP

UPDATE Students 
SET surname_encrypt = EncryptByKey(Key_GUID('SymmetricKeyDBSP'),surname)
FROM Students
CLOSE SYMMETRIC KEY SymmetricKeyDBSP

(13 rows affected)

Total execution time: 00:00:00.120

In [11]:
USE Accounting;
OPEN SYMMETRIC KEY SymmetricKeyDBSP
DECRYPTION BY CERTIFICATE CertificateDBSP
SELECT Surname_encrypt AS 'Encrypted',
CONVERT(varchar,DecryptByKey(surname_encrypt)) AS 'Decrypted',
student_id FROM Students  
CLOSE SYMMETRIC KEY SymmetricKeyDBSP

(13 rows affected)

Total execution time: 00:00:00.104

Encrypted,Decrypted,student_id
0x005F4EA2FF81BC4BB998013463577515020000003DEFCF672C889EE1EBC3C9AF7F0BB94C55F20E32A905180DFBE15AEF19F94193,ivanov,1
0x005F4EA2FF81BC4BB998013463577515020000001C0D8363837F72A424F03E79A2815649AE4A809C4652B214C7FD07F633DE66FF3D73C5AF999B181F4EEB3BF33D947EEA,petrenko,2
0x005F4EA2FF81BC4BB99801346357751502000000CC37360FCE823FD1C4DD6C7E4969787A04F18F30975607B423E0BF515F0AA492,rybnov,3
0x005F4EA2FF81BC4BB99801346357751502000000569F8A891B5978D4F029A10AD63C7D96B1D90048D15F658A2FA08D73F1FE7133,synii,4
0x005F4EA2FF81BC4BB99801346357751502000000223FBD76B44696C19785605DFE37A30F4469AAF83F182C40C8297BD39C518408,kulyba,5
0x005F4EA2FF81BC4BB9980134635775150200000069AD8334EB4D0E0A286335025BD69EB09299D5AFE2B0BBD0983EFC10620AEA0B,smyrnov,6
0x005F4EA2FF81BC4BB99801346357751502000000D938B9003C8C0F3BA220BB69F4DD09BA1ACB81EE4BFF954446B71D06E3F4C5B599C5857CF4A256D0E53E6CDF98DB3CF4,sydorenko,7
0x005F4EA2FF81BC4BB99801346357751502000000F371289C4165EC2F9AD3F544543456D3B2EC2C237C58FCC4EEAB28CE89FD1F29,rudjak,8
0x005F4EA2FF81BC4BB9980134635775150200000054E98E0883A076F12AF936010D2228101388D9155F3D3CFF412EC3E9CCA7E645,petrov,9
0x005F4EA2FF81BC4BB99801346357751502000000B1D9590E1FC489B8D9A10A7E587C7781D4608E534999693D27950BCBE925BA76328CBDF4FF859183EB6E8C2C8A29C278,symonenko,10
